In [1]:
import numpy as np
import pandas as pd
import os
import sys
import string
import copy
from collections import Counter
from operator import itemgetter
import numpy as np
from scipy.spatial.distance import cdist, pdist, squareform
import time
from scipy.cluster.hierarchy import   linkage, single, complete, average, dendrogram
from matplotlib import pyplot as plt
from PIL import Image

abspath = os.path.abspath(__file__)
dname = os.path.dirname(abspath)
os.chdir(dname)





def kmeans(data_to_cluster,k):

    row_cluster_new = [0] * (len(data_to_cluster))
    
    for iteration in range(50):
    	print iteration
        if iteration == 0:
        
            #Choose starting points randomly:
            starting_points = data_to_cluster.ix[:,[2,3]].sample(n = k )     
            row_cluster = [0]*len(data_to_cluster)
        
        else: 
            
            row_cluster = list(row_cluster_new)
   
        distance_to_cluster = [0]*len(data_to_cluster)     
        columns = 2
        
        for index, row in data_to_cluster.iterrows():    
            #print index
            
            distances = cdist(np.reshape(np.array(row[2:4]),(1,columns)) , np.array(starting_points))
            
            row_cluster_new[index] = np.argmin(distances)
            
            distance_to_cluster[index] = distances[0][np.argmin(distances)]
            
        data_to_cluster['row_cluster'] = row_cluster_new    
        
        data_to_cluster['distance_to_cluster'] = distance_to_cluster
                
        cluster_means = {}
        
        for cluster in range(k):
        
            current_cluster = data_to_cluster[data_to_cluster.row_cluster == cluster]
            
            cluster_means[cluster] = current_cluster.ix[:,[2,3]].mean()
            
        cluster_means = pd.DataFrame(cluster_means).transpose()
            
        del starting_points
        
        starting_points = cluster_means.copy(deep = True)
        
        if row_cluster == row_cluster_new:
        
            return data_to_cluster, row_cluster , starting_points, iteration
            
            #Now calculate the centroid of each cluster:
    return data_to_cluster, row_cluster, starting_points, iteration
        
    print "hello"
    

def within_cluster_sum_of_squares(clustered_data):
    
    clustered_data['squared'] = clustered_data.distance_to_cluster * clustered_data.distance_to_cluster
    
    return clustered_data.squared.sum()
    
def Silhouette(clustered_data , k):
    clustered_data_covariates = clustered_data.ix[:,[2,3]]
    distance_matrix = pdist(clustered_data_covariates)
    distance_matrix = squareform(distance_matrix)
        
    del clustered_data_covariates
        
    distance_own_cluster_index = {}
    distance_other_cluster_index = {}

    for cluster in range(k):
        distance_own_cluster_index[cluster] =  clustered_data[clustered_data['row_cluster'] == cluster].index.tolist()
        distance_other_cluster_index[cluster] =  clustered_data[clustered_data['row_cluster'] != cluster].index.tolist()
    
    average_distance_own_cluster = [0]*len(clustered_data)
    average_distance_other_cluster = [0]*len(clustered_data)
    
    for i in range(len(clustered_data)):
        current_cluster = clustered_data.row_cluster.iloc[i]
        average_distance_own_cluster[i] = np.mean(distance_matrix[i][distance_own_cluster_index[current_cluster]]) 
        average_distance_other_cluster[i] = np.mean(distance_matrix[i][distance_other_cluster_index[current_cluster]])   
    
    average_distance_own_cluster = np.array(average_distance_own_cluster)
    average_distance_other_cluster = np.array(average_distance_other_cluster)
    
    del clustered_data,distance_own_cluster_index,distance_other_cluster_index,distance_matrix
    
    S = average_distance_other_cluster - average_distance_own_cluster 
    
    maximum = np.vstack( (average_distance_own_cluster,average_distance_other_cluster) ).transpose()
    
    maximum_index = np.argmax(maximum,axis=1)
    
    maximum = [maximum[i][maximum_index[i]] for i in range(len(maximum_index)) ]
    
    maximum = np.array(maximum)
    
    S = S / maximum
    
    return np.mean(S)
    
    print 'hello'
    

def hclustering_create_clusters(z,clusters,level):
    
    cluster_to_add = len(clusters) 

    for i in range(level):
        group_1 = z[i][0]
        group_2 = z[i][1]
        clusters[clusters == group_1] = cluster_to_add
        clusters[clusters == group_2] = cluster_to_add
        cluster_to_add += 1        
   
    unique_clusters = list(set(clusters))
    
    for i in range(len(unique_clusters)):
        clusters[clusters == unique_clusters[i]] = i

    return clusters 
         
      
            
def hclustering_single_linkage(data_to_cluster,k):
    data_to_cluster_covariates = data_to_cluster.ix[:,[2,3]]
    distance_matrix = pdist(data_to_cluster_covariates)
    z = single(distance_matrix)
    dendrogram(z)
    plt.show()
    
    clusters = np.array(range(len(data_to_cluster)))
       
    level = len(data_to_cluster) - k
    
    row_cluster = hclustering_create_clusters(z,clusters,level)
    
    data_to_cluster['row_cluster'] = row_cluster      
       
    cluster_means = {}
    
    unique_clusters = list(set(row_cluster))
    
    for cluster in range(len(unique_clusters)):
    
        current_cluster = unique_clusters[cluster]
        
        current_subset = data_to_cluster[data_to_cluster.row_cluster == current_cluster]
        
        cluster_means[cluster] = current_subset.ix[:,[2,3]].mean()
         
    columns = 2
    distance_to_cluster = [0]*len(data_to_cluster)   
    
    for index, row in data_to_cluster.iterrows():    
        #print index
        
        current_cluster = row[4]
        distance_to_cluster[index] = np.linalg.norm((np.array(row[2:4]))-(np.array(cluster_means[current_cluster])))
                    
    data_to_cluster['distance_to_cluster'] = distance_to_cluster
                    
                          
    
    return data_to_cluster
    
def hclustering_complete_linkage(data_to_cluster,k):
    data_to_cluster_covariates = data_to_cluster.ix[:,[2,3]]
    distance_matrix = pdist(data_to_cluster_covariates)
    z = complete(distance_matrix)
    dendrogram(z)
    plt.show()
    
    clusters = np.array(range(len(data_to_cluster)))
       
    level = len(data_to_cluster) - k
    
    row_cluster = hclustering_create_clusters(z,clusters,level)
    
    data_to_cluster['row_cluster'] = row_cluster      
       
    cluster_means = {}
    
    unique_clusters = list(set(row_cluster))
    
    for cluster in range(len(unique_clusters)):
    
        current_cluster = unique_clusters[cluster]
        
        current_subset = data_to_cluster[data_to_cluster.row_cluster == current_cluster]
        
        cluster_means[cluster] = current_subset.ix[:,[2,3]].mean()
         
    columns = 2
    distance_to_cluster = [0]*len(data_to_cluster)   
    
    for index, row in data_to_cluster.iterrows():    
        #print index
        
        current_cluster = row[4]
        distance_to_cluster[index] = np.linalg.norm((np.array(row[2:4]))-(np.array(cluster_means[current_cluster])))
                    
    data_to_cluster['distance_to_cluster'] = distance_to_cluster
                    
                          
    
    return data_to_cluster

    
def hclustering_average_linkage(data_to_cluster,k):
    data_to_cluster_covariates = data_to_cluster.ix[:,[2,3]]
    distance_matrix = pdist(data_to_cluster_covariates)
    z = average(distance_matrix)
    dendrogram(z)
    plt.show()
    
    clusters = np.array(range(len(data_to_cluster)))
       
    level = len(data_to_cluster) - k
    
    row_cluster = hclustering_create_clusters(z,clusters,level)
    
    data_to_cluster['row_cluster'] = row_cluster      
       
    cluster_means = {}
    
    unique_clusters = list(set(row_cluster))
    
    for cluster in range(len(unique_clusters)):
    
        current_cluster = unique_clusters[cluster]
        
        current_subset = data_to_cluster[data_to_cluster.row_cluster == current_cluster]
        
        cluster_means[cluster] = current_subset.ix[:,[2,3]].mean()
         
    columns = 2
    distance_to_cluster = [0]*len(data_to_cluster)   
    
    for index, row in data_to_cluster.iterrows():    
        #print index
        
        current_cluster = row[4]
        distance_to_cluster[index] = np.linalg.norm((np.array(row[2:4]))-(np.array(cluster_means[current_cluster])))
                    
    data_to_cluster['distance_to_cluster'] = distance_to_cluster
                    
                          
    
    return data_to_cluster


def plotWCSSD(clustered_data,range_of_k , method):
    

    
    WCSSD = [0]*len(range_of_k)
    
    
    
    for i in range(len(range_of_k)):
        
        if method == 0:
            z,a,b,loops = kmeans(clustered_data.copy(),range_of_k[i])  
            del a,b,loops
        elif method == 1:      
            z = hclustering_single_linkage(clustered_data.copy(),range_of_k[i])  
        elif method == 2:      
            z = hclustering_complete_linkage(clustered_data.copy(),range_of_k[i])  
        elif method == 3:      
            z = hclustering_average_linkage(clustered_data.copy(),range_of_k[i])  

        WCSSD[i] = within_cluster_sum_of_squares(z.copy())
        del z
        
    return WCSSD
    

def nmi(clustered_data):
	
	number_of_groups = len(clustered_data.row_cluster.unique())
	
	number_of_classes = len(clustered_data[1].unique())
	
	n = len(clustered_data)
	
	numerator = 0
	
	denominator_1 = 0
	
	denominator_2 = 0
	
	for class_ in range(number_of_classes):
	
		class_count = len(clustered_data[clustered_data[1] == class_])
		
		p_c = float(class_count) / n
		
		denominator_1 += p_c*(np.log(p_c+0.001))
		
		for group in range(number_of_groups):
			
			group_count = len(clustered_data[clustered_data.row_cluster == group])
			
			p_g = float(group_count) / n
			
			if class_ == 0:
				
				denominator_2 += p_g*(np.log(p_g+0.001))
		
			p_c_g = len(clustered_data[ ( (clustered_data[1] == class_) & (clustered_data.row_cluster == group) ) ] )
			
			p_c_g = float(p_c_g) / n
			
			numerator += float(p_c_g)* ( np.log( (p_c_g / (p_c*p_g) )+0.001 ) )
			
	nmi = numerator / (-denominator_1 - denominator_2)
			
	
	return nmi


#x = data_to_cluster.ix[:,[2,3]]

print time.ctime()   

digits_embedded = pd.read_csv('digits-embedding.csv' , header = None)

KeyboardInterrupt: 